In [8]:
# !pip install --trusted-host pypi.org --trusted-host pypi.pyhton.org --trusted-host files.pythonhosted.org -U --upgrade xlrd


In [1]:
import os
import sys 
import pandas as pd
import re
from selenium import webdriver 
import win32com.client
import time
from datetime import date
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
import xlsxwriter
import concurrent.futures
from datetime import datetime
import requests

import pdfplumber

import matplotlib.pyplot as plt
import plotly.express as px

import shutil

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import StaleElementReferenceException, TimeoutException

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
#from bs4 import BeautifulSoup
import PyPDF2
from PyPDF2 import PdfReader

import xlrd

In [2]:
source_directory = r"C:\Users\bd1266\Downloads\Data Download"
backup_directory = r"C:\Users\bd1266\Downloads\Data Download\Back Up"

In [3]:
today = datetime.now().strftime("%Y-%m-%d")


In [4]:
today

'2025-02-10'

In [5]:
def start_driver():

    options = webdriver.ChromeOptions()

    #Select desired download folder.
    download_directory = "C:\\Users\\bd1266\\Downloads\\Data Download"
    prefs = {
                "download.default_directory": download_directory,
                 "download.prompt_for_download": False,
                 "download.directory_upgrade": True,
                 "safebrowsing.enabled": True,
                 "safebrowsing.disable_download_protection": True}
    options.add_experimental_option("prefs", prefs)
    driver = webdriver.Chrome(options=options)
    return driver



### BCRA

#### Principales Variables

In [72]:
try:
    driver.quit()
except:
    print("no driver open")       

In [73]:
driver = start_driver()

In [74]:
time.sleep(5)
driver.get("https://www.bcra.gob.ar/PublicacionesEstadisticas/Principales_variables.asp")
time.sleep(5)

In [75]:
list_of_names = [
    'Reservas Internacionales del BCRA', 
    'Tipo de Cambio Mayorista',  
    'Tasa de Política Monetaria', 
    'Base monetaria - Total', 
    'Circulación monetaria', 
    'Billetes y monedas en poder del público', 
    'Efectivo en entidades financieras', 
    'Depósitos de los bancos en cta. cte.', 
    'Depósitos en efectivo en las entidades financieras', 
    'M2 privado', 
    'Inflación interanual', 
    'Inflación mensual', 
    'Inflación esperada',
    'BADLAR en pesos de bancos privados (en % n.a.)',
    'BADLAR en pesos de bancos privados (en % e.a.)',
    'Tasas de interés de las operaciones de pase activas para el BCRA',
    'Tasas de interés por préstamos entre entidades financiera privadas',
    'Tasas de interés por depósitos a 30',
    'Tasa de interés de préstamos por adelantos',
    'Tasa de interés de préstamos personales',
    'Tasa de interés para uso de la Justicia']

In [76]:
cell = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '/html/body/div/div[2]/div/div/div/div/table/tbody'))
)

# Get all child elements within the specified cell
child_elements = cell.find_elements(By.XPATH, './*')  # Get all direct children
time.sleep(2)

# Count the number of child elements
item_count = len(child_elements)

results =[]
for i in range(1,item_count):
    
    item_xpath = f"/html/body/div/div[2]/div/div/div/div/table/tbody/tr[{i}]"
    try:
        kpi_name = driver.find_elements(By.XPATH, item_xpath + "/td[1]/a")[0].text
        kpi_date = driver.find_elements(By.XPATH, item_xpath + "/td[2]")[0].text
        kpi_value = driver.find_elements(By.XPATH, item_xpath + "/td[3]")[0].text
        
        result_dict = {
            "Fecha" : today,
            "Variable" : kpi_name,
            "Fecha Variable" : kpi_date,
            "Valor" : kpi_value
        }
        results.append(result_dict)
        print(result_dict)
    except:
        print("not found")

df = pd.DataFrame(results)

{'Fecha': '2025-02-06', 'Variable': 'Reservas Internacionales del BCRA (en millones de dólares - cifras provisorias sujetas a cambio de valuación)', 'Fecha Variable': '04/02/2025', 'Valor': '29.427'}
not found
{'Fecha': '2025-02-06', 'Variable': 'Tipo de Cambio Minorista ($ por USD) Comunicación B 9791 - Promedio vendedor', 'Fecha Variable': '06/02/2025', 'Valor': '1.084,69'}
{'Fecha': '2025-02-06', 'Variable': 'Tipo de Cambio Mayorista ($ por USD) Comunicación A 3500 - Referencia', 'Fecha Variable': '06/02/2025', 'Valor': '1.054,88'}
not found
not found
{'Fecha': '2025-02-06', 'Variable': '   Tasa de Política Monetaria (en % n.a.)', 'Fecha Variable': '06/02/2025', 'Valor': '29,00'}
{'Fecha': '2025-02-06', 'Variable': '   Tasa de Política Monetaria (en % e.a.)', 'Fecha Variable': '06/02/2025', 'Valor': '33,63'}
{'Fecha': '2025-02-06', 'Variable': '   TAMAR en pesos de bancos privados (en % n.a.)', 'Fecha Variable': '05/02/2025', 'Valor': '30,81'}
{'Fecha': '2025-02-06', 'Variable': '  

In [77]:
pattern = '|'.join(list_of_names)
df_ppalesvariables = df[df['Variable'].str.contains(pattern, case=False, na=False)]
df_ppalesvariables

C:\Users\bd1266\AppData\Local\Temp\ipykernel_1600\367063942.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_ppalesvariables = df[df['Variable'].str.contains(pattern, case=False, na=False)]


,Fecha,Variable,Fecha Variable,Valor
0,2025-02-06,Reservas Internacionales del BCRA (en millones...,04/02/2025,29.427
2,2025-02-06,Tipo de Cambio Mayorista ($ por USD) Comunicac...,06/02/2025,"1.054,88"
3,2025-02-06,Tasa de Política Monetaria (en % n.a.),06/02/2025,"29,00"
4,2025-02-06,Tasa de Política Monetaria (en % e.a.),06/02/2025,"33,63"
10,2025-02-06,Tasas de interés de las operaciones de pase...,06/02/2025,"33,00"
11,2025-02-06,Tasas de interés por préstamos entre entida...,05/02/2025,"34,36"
12,2025-02-06,Tasas de interés por depósitos a 30 días de...,05/02/2025,"28,51"
13,2025-02-06,Tasa de interés de préstamos por adelantos ...,05/02/2025,"37,01"
14,2025-02-06,Tasa de interés de préstamos personales,05/02/2025,"68,36"
15,2025-02-06,Tasa de interés para uso de la Justicia - Comu...,06/02/2025,"18.352,1381"


### Informe monetario diario

In [333]:
try:
    driver.quit()
except:
    print("no driver open")       

In [334]:
driver = start_driver()

In [335]:
time.sleep(5)
driver.get("https://www.bcra.gob.ar/PublicacionesEstadisticas/Informe_monetario_diario.asp")
time.sleep(5)

In [336]:
excel_link = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '/html/body/div/div[2]/div/div/div[2]/div/ul[1]/li[2]/a'))
)
excel_link.click()
time.sleep(10)

In [337]:
file_to_rename = None
for filename in os.listdir(source_directory):
    if "infomondia" in filename:
        print("its here")
        file_to_rename = filename
        break  

if file_to_rename is None:
    print("No file containing 'infomondia' found.")
else:
    
    new_filename = f"{today}_{file_to_rename}"
    old_file_path = os.path.join(source_directory, file_to_rename)
    new_file_path = os.path.join(source_directory, new_filename)
    os.rename(old_file_path, new_file_path)
    print(f"Renamed '{file_to_rename}' to '{new_filename}'.")


its here
Renamed 'infomondia.xls' to '2025-02-10_infomondia.xls'.


#### DATOS | DATA sheet

In [338]:
def extract_table(file_path, sheet_name, start_col, end_col, start_row):
    # Construct the column range string
    col_range = f"{start_col}:{end_col}"
    
    # Read the specific columns and rows from the sheet
    df = pd.read_excel(file_path, sheet_name=sheet_name, usecols=col_range, skiprows=start_row-1)
    # Drop rows and columns that are completely empty
    df.dropna(how='all', inplace=True)
    df.dropna(axis=1, how='all', inplace=True)
    
    # Remove any leading or trailing whitespace from column names
    df.columns = df.columns.str.strip()
    
    # Replace any remaining NaN values with an empty string
    df.fillna('', inplace=True)
    
    return df

    

In [339]:
sheet_name = 'DATOS | DATA'
start_col = 'AZ'  # Change this to 'AX' or any other column as needed
end_col = 'BF'
start_row = 20

# Extract the table
df_factores_variacion = extract_table(new_file_path, sheet_name, start_col, end_col, start_row)

df_factores_variacion = df_factores_variacion[~df_factores_variacion.apply(lambda row: row.astype(str).str.contains('gra').any(), axis=1)]
df_factores_variacion = df_factores_variacion[df_factores_variacion['Compras de divisas al sector privado y otros'] != '' ]
df_factores_variacion.reset_index(drop=True, inplace=True)

# Display the cleaned dataframe
df_factores_variacion

,Fecha.11,Compras de divisas al sector privado y otros,Tesoro Nacional (venta neta de divisas y otras op.),"Pases, LEBAC, LELIQ, NOTALIQ, Redescuentos (inc. intereses)",Operaciones con LEFI,Otros,Base monetaria
0,2017-01-02 00:00:00,11.173738,4.991505,80.449171,0,-0.223774,96.390639
1,2017-01-03 00:00:00,11.173738,5.052279,83.597743,0,-0.102714,99.721046
2,2017-01-04 00:00:00,11.173738,5.141042,74.63813,0,-0.056575,90.896335
3,2017-01-05 00:00:00,11.173738,4.952822,69.766877,0,-0.019488,85.873949
4,2017-01-06 00:00:00,11.173738,5.122436,51.431062,0,0.027268,67.754504
...,...,...,...,...,...,...,...
1965,2025-01-29 00:00:00,1627.726975,-1453.219953,40.568782,-246.822694,-499.980612,-531.727502
1966,2025-01-30 00:00:00,1576.282975,-1790.950043,383.712543,-1681.671747,-466.165336,-1978.791608
1967,2025-01-31 00:00:00,1469.837075,417.590171,160.107101,-2967.642394,-493.774106,-1413.882155
1968,2025-02-03 00:00:00,1613.58705,412.064269,-114.093131,-1490.354129,-495.731591,-74.527532


In [340]:
sheet_name = 'DATOS | DATA'
start_col = 'R'  # Change this to 'AX' or any other column as needed
end_col = 'T'
start_row = 20

# Extract the table
df_agregados_monetarios = extract_table(new_file_path, sheet_name, start_col, end_col, start_row)

df_agregados_monetarios = df_agregados_monetarios[~df_agregados_monetarios.apply(lambda row: row.astype(str).str.contains('gra').any(), axis=1)]
#df_agregados_monetarios = df_agregados_monetarios[df_agregados_monetarios['Compras de divisas al sector privado y otros'] != '' ]
df_agregados_monetarios.reset_index(drop=True, inplace=True)

# Display the cleaned dataframe
df_agregados_monetarios

,Fecha.3,Base Monetaria,M2 Privado
0,2017-01-01 00:00:00,26.95751,24.514189
1,2017-01-02 00:00:00,27.445507,24.856786
2,2017-01-03 00:00:00,27.89968,25.236133
3,2017-01-04 00:00:00,28.330789,25.594812
4,2017-01-05 00:00:00,28.722517,25.982199
...,...,...,...
2953,2025-02-01 00:00:00,194.059387,80.143228
2954,2025-02-02 00:00:00,192.729862,80.225449
2955,2025-02-03 00:00:00,191.8446,80.305811
2956,2025-02-04 00:00:00,191.315934,80.368755


In [341]:
sheet_name = 'DATOS | DATA'
start_col = 'AS'  # Change this to 'AX' or any other column as needed
end_col = 'AU'
start_row = 20

# Extract the table
df_depositos_extranjera = extract_table(new_file_path, sheet_name, start_col, end_col, start_row)

df_depositos_extranjera = df_depositos_extranjera[~df_depositos_extranjera.apply(lambda row: row.astype(str).str.contains('gra').any(), axis=1)]
#df_depositos_extranjera = df_depositos_extranjera[df_depositos_extranjera['Compras de divisas al sector privado y otros'] != '' ]
df_depositos_extranjera.reset_index(drop=True, inplace=True)

# Display the cleaned dataframe
df_depositos_extranjera

,Fecha.9,Préstamos al sector privado,Depósitos del sector privado
0,2017-01-01 00:00:00,215.646337,128.614592
1,2017-01-02 00:00:00,215.561926,127.777348
2,2017-01-03 00:00:00,215.655419,126.910512
3,2017-01-04 00:00:00,215.86125,126.024665
4,2017-01-05 00:00:00,215.904165,125.109323
...,...,...,...
2953,2025-02-01 00:00:00,227.511608,99.707706
2954,2025-02-02 00:00:00,227.835214,99.57869
2955,2025-02-03 00:00:00,228.109145,99.408953
2956,2025-02-04 00:00:00,228.54373,99.227198


#### CUADRO 1 | TABLE 1 sheet

In [342]:
def extract_table2(file_path, sheet_name, start_col, end_col, start_row, end_row=None):
    # Construct the column range string
    col_range = f"{start_col}:{end_col}"
    
    # Read the specific columns and rows from the sheet
    if end_row == None:
        df = pd.read_excel(file_path, sheet_name=sheet_name, usecols=col_range, skiprows=start_row-1)
    # Drop rows and columns that are completely empty
    else:
        nrows = end_row - start_row + 1
        df = pd.read_excel(file_path, sheet_name=sheet_name, usecols=col_range, skiprows=start_row-1, nrows=nrows)
    df.dropna(how='all', inplace=True)
    df.dropna(axis=1, how='all', inplace=True)
    
    # Remove any leading or trailing whitespace from column names
    def is_date(string):
        try:
            pd.to_datetime(string)
            return True
        except ValueError:
            return False

# Verify if the column header is a date and format it as '2025-02-06' and convert it to text
    df.columns = [pd.to_datetime(col).strftime('%Y-%m-%d') if is_date(col) else col.strip() for col in df.columns]
    
    # Replace any remaining NaN values with an empty string
    df.fillna('', inplace=True)
    
    return df

sheet_name = 'CUADRO 1 | TABLE 1'
start_col = 'A'
end_col = 'H'
start_row = 4

# Extract the table
df_cuadro1 = extract_table2(new_file_path, sheet_name, start_col, end_col, start_row)


In [343]:
filtered_cuadro1 = df_cuadro1.loc[:df_cuadro1[df_cuadro1['Variable'] == 'Total en pesos'].index[0]]
col_to_drop = df_cuadro1.columns[1]
filtered_cuadro1 = filtered_cuadro1.drop(columns=[col_to_drop])

In [344]:
def correct_multi_index(df):
    current_header = ""
    first_col = df.columns[0]
    date_header =  df.columns[1]
    for index, row in df.iterrows():
        if row[date_header] =='':
            current_header = row[first_col]
            df.drop(index=index, inplace=True)
        else:
            df.at[index, first_col] = f"{current_header}_{row[first_col]}" if current_header else row[first_col]
    return df

In [345]:
df_tasas_interés_nominal_anual = correct_multi_index(filtered_cuadro1)
df_tasas_interés_nominal_anual

,Variable,2025-02-06,Promedio 1 7 días antes,Promedio 1 30 días antes
0,Tasa de política monetaria 8,29,30.8,32
2,Pases Activos 1 día,33,34.8,36
3,Call en pesos - Operaciones h/15 días,33.15,35.596,36.35
5,Plazo Fijo 30 días_Pesos,29.275617,30.280114,32.024949
6,Plazo Fijo 30 días_Dólares,0.365323,0.527973,0.481265
8,TAMAR 13_Total,30.4375,32.1,34.15
9,TAMAR 13_Bancos Privados,30.625,32.45,34.4375
11,BADLAR 2_Total,29.125,30.1,31.875
12,BADLAR 2_Bancos Privados,29.4375,30.6125,31.9
14,TM20 7_Total,29.8125,30.9625,32.8625


In [346]:
# Filter the DataFrame to include only the rows from 'Variable' column that contain 'Saldos, en millones, en la moneda de origen'
filtered_cuadro2 = df_cuadro1.loc[df_cuadro1[df_cuadro1['Variable'].str.contains('Saldos, en millones, en la moneda de origen')].index[0]:]

# Drop the column to be removed
col_to_drop = df_cuadro1.columns[1]
filtered_cuadro2 = filtered_cuadro2.drop(columns=[col_to_drop])

# Reset the index
filtered_cuadro2.reset_index(drop=True, inplace=True)

# Verify if the column header is a date and format it as '2025-02-06' and convert it to text
def is_date(string):
    try:
        pd.to_datetime(string)
        return True
    except ValueError:
        return False

date_col = filtered_cuadro2.iloc[1][1].strftime('%Y-%m-%d') if is_date(filtered_cuadro2.iloc[1][1]) else filtered_cuadro2.iloc[1][1].strip()
filtered_cuadro2.columns = [filtered_cuadro2.iloc[0][0], date_col.strip(), filtered_cuadro2.iloc[1][2], filtered_cuadro2.iloc[1][3]]

# Drop the first row as it is now the header
filtered_cuadro2 = filtered_cuadro2[1:]

# Reset the index again
filtered_cuadro2.reset_index(drop=True, inplace=True)

# Filter the dataframe to include only the rows from 'Variable' column that contain 'Base monetaria'
header0 = filtered_cuadro2.columns[0]
filtered_cuadro2 = filtered_cuadro2.loc[filtered_cuadro2[filtered_cuadro2[header0].str.contains('Base monetaria')].index[0]:]


filtered_cuadro2


,"Saldos, en millones, en la moneda de origen",2025-02-04,30 días antes,12 meses antes
1,Base monetaria,31793497.531569,31660479.415815,10613776.558366
2,Circulación monetaria,20167041.388919,20064706.90964,7537232.129241
3,Billetes y Monedas en poder del público,18055246.141568,17746868.662289,6736703.881891
4,Billetes y Monedas en ent. financieras 4,2111795,2317838,800528
5,Cheques Cancelatorios,0.24735,0.24735,0.24735
6,Cuenta corriente en el BCRA,11626456.142651,11595772.506175,3076544.429125
7,Cuenta corriente en moneda extranjera 5,13812.683606,14381.311126,10068.833
8,Efectivo en ent. financieras en moneda extranj...,7826,8917,5674
9,Cheques Cancelatorios en moneda extranjera y C...,,,
10,,19,20,21


In [347]:
saldos_mill_moneda_origen = filtered_cuadro2

In [348]:
sheet_name = 'CUADRO 1 | TABLE 1'
start_col = 'I'
end_col = 'O'
start_row = 4
end_row = 26
# Extract the table
df_cuadro3 = extract_table2(new_file_path, sheet_name, start_col, end_col, start_row, end_row)

In [350]:
otros_indicadores_df = correct_multi_index(df_cuadro3)

In [351]:
sheet_name = 'CUADRO 1 | TABLE 1'
start_col = 'A'
end_col = 'O'
start_row = 28
end_row = 32
# Extract the table
df_tasas_interes_prestamos_pesos = extract_table2(new_file_path, sheet_name, start_col, end_col, start_row, end_row)

In [353]:
columns = df_tasas_interes_prestamos_pesos.columns.tolist()  
columns[0] = 'Tasas de interés de préstamos otorgados en pesos'
df_tasas_interes_prestamos_pesos.columns = columns  


In [354]:
df_tasas_interes_prestamos_pesos

,Tasas de interés de préstamos otorgados en pesos,Adelantos en cta. cte.,Doc. a sola firma,Hipotecarios,Prendarios,Personales,Tarjetas
0,Enero 2025,46.07,36.48,33.63,29.66,71.10,83.33
1,Diciembre 2024,48.44,35.99,32.95,34.23,70.27,84.56
2,Noviembre 2024,52.60,38.23,33.75,38.05,70.70,83.86


In [355]:
sheet_name = 'CUADRO 1 | TABLE 1'
start_col = 'I'
end_col = 'O'
start_row = 36
end_row = 66
# Extract the table
df_variacion_base_monetaria = extract_table2(new_file_path, sheet_name, start_col, end_col, start_row, end_row)

In [356]:
columns = df_variacion_base_monetaria.columns.tolist()  
columns[0] = 'Variación de la Base Monetaria y las Reservas Internacionales'
df_variacion_base_monetaria.columns = columns  


In [357]:
df_variacion_base_monetaria

,Variación de la Base Monetaria y las Reservas Internacionales,Diaria (mill.),Mensual (mill.),Año 2025,Interanual
0,Base monetaria,598607.918472,1546900.270375,0.069459,1.995493
1,Circulación monetaria,19048.664,48534.664,0.008944,1.675656
2,Billetes y Monedas en poder del público,12466.664,65429.664,0.014667,1.680131
3,Billetes y Monedas en ent. financieras 4,6582.0,-16895.0,-0.037479,1.638003
4,Cheques Cancelatorios,0.0,0.0,0,0
5,Cuenta corriente en el BCRA,579559.254472,1498365.606375,0.193645,2.779063
6,Factores de variación,,,Año 2025 (mill.),12 meses (mill.)
8,Variación Base monetaria,598607.918472,1546900.270375,2064922.644441,21350980.678043
9,Compra de divisas al sector privado y otros,17913.75,189026.25,1868827.225,15478974.618751
10,Compra de divisas al Tesoro Nacional,0.0,0.0,-1041250,-15249118.771109


In [358]:
try:
    shutil.move(new_file_path, backup_directory)
    print(f"Moved '{new_filename}' to '{backup_directory}'.")
except Exception as e:
    print(f"Error moving the file: {e}")

Moved '2025-02-10_infomondia.xls' to 'C:\Users\bd1266\Downloads\Data Download\Back Up'.


In [359]:
driver.quit()

In [360]:
#Results dfs

df_factores_variacion, 
df_agregados_monetarios,
df_depositos_extranjera,
df_tasas_interés_nominal_anual,
saldos_mill_moneda_origen,
otros_indicadores_df,
df_tasas_interes_prestamos_pesos,
df_variacion_base_monetaria

,Variación de la Base Monetaria y las Reservas Internacionales,Diaria (mill.),Mensual (mill.),Año 2025,Interanual
0,Base monetaria,598607.918472,1546900.270375,0.069459,1.995493
1,Circulación monetaria,19048.664,48534.664,0.008944,1.675656
2,Billetes y Monedas en poder del público,12466.664,65429.664,0.014667,1.680131
3,Billetes y Monedas en ent. financieras 4,6582.0,-16895.0,-0.037479,1.638003
4,Cheques Cancelatorios,0.0,0.0,0,0
5,Cuenta corriente en el BCRA,579559.254472,1498365.606375,0.193645,2.779063
6,Factores de variación,,,Año 2025 (mill.),12 meses (mill.)
8,Variación Base monetaria,598607.918472,1546900.270375,2064922.644441,21350980.678043
9,Compra de divisas al sector privado y otros,17913.75,189026.25,1868827.225,15478974.618751
10,Compra de divisas al Tesoro Nacional,0.0,0.0,-1041250,-15249118.771109


### Trading Economics

#### Bonds

In [109]:
try:
    driver.quit()
except:
    print("no driver open")       

In [110]:
driver = start_driver()

In [111]:
time.sleep(5)
driver.get("https://tradingeconomics.com/bonds")
time.sleep(15)

In [112]:
table = driver.find_elements(By.XPATH, '/html/body/form/div[4]/div/div/div[1]/div/table/tbody')[0]
rows = table.find_elements(By.TAG_NAME, 'tr')
data = []

# Iterate through each row and extract data
for row in rows:
    columns = row.find_elements(By.TAG_NAME, 'td')
    
    # Extract data from each column based on the specified headers
    major = columns[1].text.strip()
    yield_value = columns[2].text.strip()
    day = columns[3].text.strip()
    weekly = columns[4].text.strip()
    monthly = columns[5].text.strip()
    ytd = columns[6].text.strip()
    yoy = columns[7].text.strip()
    date = today
    data.append([major, yield_value, day, weekly, monthly, ytd, yoy, date])
    # Print the extracted data
    print(f"Major: {major}, Yield: {yield_value}, Day: {day}, Weekly: {weekly}, Monthly: {monthly}, YTD: {ytd}, YoY: {yoy}, Date: {date}")

df_trading_economics_bonds = pd.DataFrame(data, columns=['Major', 'Yield', 'Day', 'Weekly', 'Monthly', 'YTD', 'YoY', 'Date'])

Major: United States, Yield: 4.4390, Day: 0.0070, Weekly: -0.08%, Monthly: -0.25%, YTD: -0.14%, YoY: 0.33%, Date: 2025-02-06
Major: United Kingdom, Yield: 4.4900, Day: 0.0487, Weekly: -0.08%, Monthly: -0.20%, YTD: -0.08%, YoY: 0.50%, Date: 2025-02-06
Major: Japan, Yield: 1.2670, Day: 0.0165, Weekly: 0.05%, Monthly: 0.14%, YTD: 0.17%, YoY: 0.56%, Date: 2025-02-06
Major: Australia, Yield: 4.3840, Day: 0.0010, Weekly: -0.01%, Monthly: -0.09%, YTD: 0.02%, YoY: 0.29%, Date: 2025-02-06
Major: Germany, Yield: 2.3670, Day: 0.0080, Weekly: -0.15%, Monthly: -0.12%, YTD: 0.01%, YoY: 0.07%, Date: 2025-02-06
Major: Brazil, Yield: 14.7970, Day: 0.1570, Weekly: -0.21%, Monthly: 0.21%, YTD: -0.35%, YoY: 4.18%, Date: 2025-02-06
Major: Russia, Yield: 16.3600, Day: 0.2700, Weekly: -0.01%, Monthly: 1.18%, YTD: 1.14%, YoY: 3.96%, Date: 2025-02-06
Major: India, Yield: 6.7170, Day: 0.0060, Weekly: -0.01%, Monthly: -0.06%, YTD: -0.08%, YoY: -0.36%, Date: 2025-02-06
Major: Canada, Yield: 2.9820, Day: 0.0330, W

In [113]:
df_trading_economics_bonds

,Major,Yield,Day,Weekly,Monthly,YTD,YoY,Date
0,United States,4.4390,0.0070,-0.08%,-0.25%,-0.14%,0.33%,2025-02-06
1,United Kingdom,4.4900,0.0487,-0.08%,-0.20%,-0.08%,0.50%,2025-02-06
2,Japan,1.2670,0.0165,0.05%,0.14%,0.17%,0.56%,2025-02-06
3,Australia,4.3840,0.0010,-0.01%,-0.09%,0.02%,0.29%,2025-02-06
4,Germany,2.3670,0.0080,-0.15%,-0.12%,0.01%,0.07%,2025-02-06
5,Brazil,14.7970,0.1570,-0.21%,0.21%,-0.35%,4.18%,2025-02-06
6,Russia,16.3600,0.2700,-0.01%,1.18%,1.14%,3.96%,2025-02-06
7,India,6.7170,0.0060,-0.01%,-0.06%,-0.08%,-0.36%,2025-02-06
8,Canada,2.9820,0.0330,-0.14%,-0.32%,-0.25%,-0.50%,2025-02-06
9,Italy,3.4440,0.0045,-0.15%,-0.18%,-0.08%,-0.45%,2025-02-06


#### Stocks

In [114]:
try:
    driver.quit()
except:
    print("no driver open")       

In [115]:
driver = start_driver()

In [116]:
time.sleep(5)
driver.get("https://tradingeconomics.com/stocks")
time.sleep(15)

In [117]:
table = driver.find_elements(By.XPATH, '/html/body/form/div[4]/div/div/div[1]/div/table/tbody')[0]
rows = table.find_elements(By.TAG_NAME, 'tr')
data = []

# Iterate through each row and extract data
for row in rows:
    columns = row.find_elements(By.TAG_NAME, 'td')
    
    # Extract data from each column based on the specified headers
    major = columns[1].text.strip()
    yield_value = columns[2].text.strip()
    day = columns[3].text.strip()
    weekly = columns[4].text.strip()
    monthly = columns[5].text.strip()
    ytd = columns[6].text.strip()
    yoy = columns[7].text.strip()
    date = today
    data.append([major, yield_value, day, weekly, monthly, ytd, yoy, date])
    # Print the extracted data
    print(f"Major: {major}, Yield: {yield_value}, Day: {day}, Weekly: {weekly}, Monthly: {monthly}, YTD: {ytd}, YoY: {yoy}, Date: {date}")

df_trading_economics_stocks = pd.DataFrame(data, columns=['Major', 'Yield', 'Day', 'Weekly', 'Monthly', 'YTD', 'YoY', 'Date'])

Major: US500, Yield: 6074.92, Day: 13.44, Weekly: 0.22%, Monthly: 0.07%, YTD: 2.82%, YoY: 3.30%, Date: 2025-02-06
Major: US30, Yield: 44706, Day: 168, Weekly: -0.37%, Monthly: -0.38%, YTD: 5.13%, YoY: 5.09%, Date: 2025-02-06
Major: US100, Yield: 21742, Day: 84, Weekly: 0.39%, Monthly: 1.07%, YTD: 2.67%, YoY: 3.46%, Date: 2025-02-06
Major: JP225, Yield: 38920, Day: 89, Weekly: 0.23%, Monthly: -1.50%, YTD: -2.90%, YoY: -2.44%, Date: 2025-02-06
Major: GB100, Yield: 8727, Day: 104, Weekly: 1.21%, Monthly: 0.93%, YTD: 5.85%, YoY: 6.78%, Date: 2025-02-06
Major: DE40, Yield: 21902, Day: 316, Weekly: 1.47%, Monthly: 0.81%, YTD: 7.68%, YoY: 10.01%, Date: 2025-02-06
Major: FR40, Yield: 8008, Day: 116, Weekly: 1.47%, Monthly: 0.83%, YTD: 6.92%, YoY: 8.49%, Date: 2025-02-06
Major: IT40, Yield: 37122, Day: 540, Weekly: 1.48%, Monthly: 1.90%, YTD: 6.25%, YoY: 8.59%, Date: 2025-02-06
Major: ES35, Yield: 12731, Day: 194, Weekly: 1.55%, Monthly: 2.51%, YTD: 7.78%, YoY: 9.80%, Date: 2025-02-06
Major: AS

In [118]:
df_trading_economics_stocks

,Major,Yield,Day,Weekly,Monthly,YTD,YoY,Date
0,US500,6074.92,13.44,0.22%,0.07%,2.82%,3.30%,2025-02-06
1,US30,44706,168,-0.37%,-0.38%,5.13%,5.09%,2025-02-06
2,US100,21742,84,0.39%,1.07%,2.67%,3.46%,2025-02-06
3,JP225,38920,89,0.23%,-1.50%,-2.90%,-2.44%,2025-02-06
4,GB100,8727,104,1.21%,0.93%,5.85%,6.78%,2025-02-06
5,DE40,21902,316,1.47%,0.81%,7.68%,10.01%,2025-02-06
6,FR40,8008,116,1.47%,0.83%,6.92%,8.49%,2025-02-06
7,IT40,37122,540,1.48%,1.90%,6.25%,8.59%,2025-02-06
8,ES35,12731,194,1.55%,2.51%,7.78%,9.80%,2025-02-06
9,ASX200,8521,104,1.23%,0.32%,2.84%,4.43%,2025-02-06


#### Currencies

In [119]:
try:
    driver.quit()
except:
    print("no driver open")       

In [120]:
driver = start_driver()

In [121]:
time.sleep(5)
driver.get("https://tradingeconomics.com/currencies")
time.sleep(15)

In [122]:
table = driver.find_elements(By.XPATH, '/html/body/form/div[4]/div/div/div[1]/div/table/tbody')[0]
rows = table.find_elements(By.TAG_NAME, 'tr')
data = []

# Iterate through each row and extract data
for row in rows:
    columns = row.find_elements(By.TAG_NAME, 'td')
    
    # Extract data from each column based on the specified headers
    Major = columns[0].text.strip()
    price = columns[1].text.strip()
    day = columns[2].text.strip()
    day_percent = columns[3].text.strip()
    weekly = columns[4].text.strip()
    monthly = columns[5].text.strip()
    ytd = columns[6].text.strip()
    yoy = columns[7].text.strip()
    date = today
    data.append([Major, price, day, day_percent, weekly, monthly, ytd, yoy, date])
    # Print the extracted data
    print(f"Major: {Major}, price: {price}, Day: {day}, day_percent: {day_percent}, Weekly: {weekly}, Monthly: {monthly}, YTD: {ytd}, YoY: {yoy},  Date: {date}")

df_trading_economics_currencies = pd.DataFrame(data, columns=['Major', 'price', 'Day', 'day_percent', 'Weekly', 'Monthly', 'YTD', 'YoY', 'Date'])

Major: , price: EURUSD, Day: 1.03853, day_percent: 0.00170, Weekly: -0.17%, Monthly: -0.05%, YTD: 0.43%, YoY: 0.26%,  Date: 2025-02-06
Major: , price: GBPUSD, Day: 1.24366, day_percent: 0.00690, Weekly: -0.55%, Monthly: 0.17%, YTD: -0.33%, YoY: -0.66%,  Date: 2025-02-06
Major: , price: AUDUSD, Day: 0.62843, day_percent: 0.00007, Weekly: -0.01%, Monthly: 1.21%, YTD: 0.86%, YoY: 1.53%,  Date: 2025-02-06
Major: , price: NZDUSD, Day: 0.56755, day_percent: 0.00000, Weekly: 0.00%, Monthly: 0.57%, YTD: 1.28%, YoY: 1.48%,  Date: 2025-02-06
Major: , price: USDJPY, Day: 151.467, day_percent: 1.157, Weekly: -0.76%, Monthly: -1.83%, YTD: -4.18%, YoY: -3.74%,  Date: 2025-02-06
Major: , price: USDCNY, Day: 7.28795, day_percent: 0.00525, Weekly: 0.07%, Monthly: -0.07%, YTD: -0.74%, YoY: -0.68%,  Date: 2025-02-06
Major: , price: USDCHF, Day: 0.90490, day_percent: 0.00305, Weekly: 0.34%, Monthly: -0.56%, YTD: -0.54%, YoY: -0.28%,  Date: 2025-02-06
Major: , price: USDCAD, Day: 1.43065, day_percent: 0.00

In [123]:
df_trading_economics_currencies

,Major,price,Day,day_percent,Weekly,Monthly,YTD,YoY,Date
0,,EURUSD,1.03853,0.00170,-0.17%,-0.05%,0.43%,0.26%,2025-02-06
1,,GBPUSD,1.24366,0.00690,-0.55%,0.17%,-0.33%,-0.66%,2025-02-06
2,,AUDUSD,0.62843,0.00007,-0.01%,1.21%,0.86%,1.53%,2025-02-06
3,,NZDUSD,0.56755,0.00000,0.00%,0.57%,1.28%,1.48%,2025-02-06
4,,USDJPY,151.467,1.157,-0.76%,-1.83%,-4.18%,-3.74%,2025-02-06
5,,USDCNY,7.28795,0.00525,0.07%,-0.07%,-0.74%,-0.68%,2025-02-06
6,,USDCHF,0.90490,0.00305,0.34%,-0.56%,-0.54%,-0.28%,2025-02-06
7,,USDCAD,1.43065,0.00060,-0.04%,-1.23%,-0.46%,-0.49%,2025-02-06
8,,USDMXN,20.4694,0.1160,-0.56%,-1.26%,0.68%,-1.82%,2025-02-06
9,,USDINR,87.5530,0.2122,0.24%,1.02%,2.04%,2.31%,2025-02-06


#### Commodities

In [124]:
try:
    driver.quit()
except:
    print("no driver open")       

In [125]:
driver = start_driver()

In [126]:
time.sleep(5)
driver.get("https://tradingeconomics.com/commodities")
time.sleep(15)

##### Energy

In [127]:
table = driver.find_elements(By.XPATH, '/html/body/form/div[4]/div/div/div[1]/div/table/tbody')[0]
rows = table.find_elements(By.TAG_NAME, 'tr')
data = []

# Iterate through each row and extract data
for row in rows:
    columns = row.find_elements(By.TAG_NAME, 'td')
    
    # Extract data from each column based on the specified headers
    Energy = columns[0].text.strip().split("\n")[0]
    price = columns[1].text.strip()
    day = columns[2].text.strip()
    day_percent = columns[3].text.strip()
    weekly = columns[4].text.strip()
    monthly = columns[5].text.strip()
    ytd = columns[6].text.strip()
    yoy = columns[7].text.strip()
    date = today
    data.append([Energy, price, day, day_percent, weekly, monthly, ytd, yoy, date])
    # Print the extracted data
    print(f"Energy: {Energy}, price: {price}, Day: {day}, day_percent: {day_percent}, Weekly: {weekly}, Monthly: {monthly}, YTD: {ytd}, YoY: {yoy},  Date: {date}")

df_trading_economics_Energy = pd.DataFrame(data, columns=['Energy', 'price', 'Day', 'day_percent', 'Weekly', 'Monthly', 'YTD', 'YoY', 'Date'])

Energy: Crude Oil, price: 70.517, Day: 0.513, day_percent: -0.72%, Weekly: -3.04%, Monthly: -5.03%, YTD: -1.68%, YoY: -4.56%,  Date: 2025-02-06
Energy: Brent, price: 74.261, Day: 0.348, day_percent: -0.47%, Weekly: -2.16%, Monthly: -3.63%, YTD: -0.52%, YoY: -5.88%,  Date: 2025-02-06
Energy: Natural gas, price: 3.3766, Day: 0.0166, day_percent: 0.49%, Weekly: 10.75%, Monthly: -2.16%, YTD: -7.11%, YoY: 70.43%,  Date: 2025-02-06
Energy: Gasoline, price: 2.0736, Day: 0.0227, day_percent: 1.11%, Weekly: -0.40%, Monthly: 1.86%, YTD: 3.04%, YoY: -16.47%,  Date: 2025-02-06
Energy: Heating Oil, price: 2.4071, Day: 0.0209, day_percent: 0.88%, Weekly: -2.76%, Monthly: 1.48%, YTD: 3.88%, YoY: -12.23%,  Date: 2025-02-06
Energy: Coal, price: 111.25, Day: 1.25, day_percent: -1.11%, Weekly: -3.09%, Monthly: -7.10%, YTD: -11.18%, YoY: -6.51%,  Date: 2025-02-06
Energy: TTF Gas, price: 54.14, Day: 0.75, day_percent: 1.41%, Weekly: 5.44%, Monthly: 12.97%, YTD: 7.30%, YoY: 92.07%,  Date: 2025-02-06
Energy:

In [128]:
df_trading_economics_Energy

,Energy,price,Day,day_percent,Weekly,Monthly,YTD,YoY,Date
0,Crude Oil,70.517,0.513,-0.72%,-3.04%,-5.03%,-1.68%,-4.56%,2025-02-06
1,Brent,74.261,0.348,-0.47%,-2.16%,-3.63%,-0.52%,-5.88%,2025-02-06
2,Natural gas,3.3766,0.0166,0.49%,10.75%,-2.16%,-7.11%,70.43%,2025-02-06
3,Gasoline,2.0736,0.0227,1.11%,-0.40%,1.86%,3.04%,-16.47%,2025-02-06
4,Heating Oil,2.4071,0.0209,0.88%,-2.76%,1.48%,3.88%,-12.23%,2025-02-06
5,Coal,111.25,1.25,-1.11%,-3.09%,-7.10%,-11.18%,-6.51%,2025-02-06
6,TTF Gas,54.14,0.75,1.41%,5.44%,12.97%,7.30%,92.07%,2025-02-06
7,UK Gas,132.9472,1.9607,1.50%,4.75%,12.92%,6.03%,92.73%,2025-02-06
8,Ethanol,1.7675,0.0225,-1.26%,-0.28%,5.21%,4.59%,12.22%,2025-02-06
9,Naphtha,641.11,7.58,-1.17%,-1.17%,-0.80%,4.84%,1.28%,2025-02-06


##### Metals

In [129]:
table = driver.find_elements(By.XPATH, '/html/body/form/div[4]/div/div/div[2]/div/table/tbody')[0]
rows = table.find_elements(By.TAG_NAME, 'tr')
data = []

# Iterate through each row and extract data
for row in rows:
    columns = row.find_elements(By.TAG_NAME, 'td')
    
    # Extract data from each column based on the specified headers
    Metal = columns[0].text.strip().split("\n")[0]
    price = columns[1].text.strip()
    day = columns[2].text.strip()
    day_percent = columns[3].text.strip()
    weekly = columns[4].text.strip()
    monthly = columns[5].text.strip()
    ytd = columns[6].text.strip()
    yoy = columns[7].text.strip()
    date = today
    data.append([Metal, price, day, day_percent, weekly, monthly, ytd, yoy, date])
    # Print the extracted data
    print(f"Metal: {Metal}, price: {price}, Day: {day}, day_percent: {day_percent}, Weekly: {weekly}, Monthly: {monthly}, YTD: {ytd}, YoY: {yoy},  Date: {date}")

df_trading_economics_Metal = pd.DataFrame(data, columns=['Metal', 'price', 'Day', 'day_percent', 'Weekly', 'Monthly', 'YTD', 'YoY', 'Date'])

Metal: Gold, price: 2856.09, Day: 12.91, day_percent: -0.45%, Weekly: 2.14%, Monthly: 7.79%, YTD: 8.83%, YoY: 40.41%,  Date: 2025-02-06
Metal: Silver, price: 32.225, Day: 0.087, day_percent: -0.27%, Weekly: 1.89%, Monthly: 7.10%, YTD: 11.58%, YoY: 45.26%,  Date: 2025-02-06
Metal: Copper, price: 4.4475, Day: 0.0130, day_percent: 0.29%, Weekly: 3.72%, Monthly: 6.90%, YTD: 11.58%, YoY: 19.06%,  Date: 2025-02-06
Metal: Steel, price: 3286.00, Day: 28.00, day_percent: 0.86%, Weekly: 0.86%, Monthly: 2.75%, YTD: -0.73%, YoY: -15.40%,  Date: 2025-02-06
Metal: Iron Ore CNY, price: 817.50, Day: 13.00, day_percent: 1.62%, Weekly: 1.62%, Monthly: 8.57%, YTD: 4.94%, YoY: -13.40%,  Date: 2025-02-06
Metal: Lithium, price: 77300, Day: 350, day_percent: -0.45%, Weekly: -0.45%, Monthly: 2.45%, YTD: 3.00%, YoY: -20.72%,  Date: 2025-02-06
Metal: Platinum, price: 1011.30, Day: 4.30, day_percent: -0.42%, Weekly: -0.51%, Monthly: 5.04%, YTD: 13.12%, YoY: 14.98%,  Date: 2025-02-06
Metal: Iron Ore, price: 104.8

In [130]:
df_trading_economics_Metal

,Metal,price,Day,day_percent,Weekly,Monthly,YTD,YoY,Date
0,Gold,2856.09,12.91,-0.45%,2.14%,7.79%,8.83%,40.41%,2025-02-06
1,Silver,32.225,0.087,-0.27%,1.89%,7.10%,11.58%,45.26%,2025-02-06
2,Copper,4.4475,0.0130,0.29%,3.72%,6.90%,11.58%,19.06%,2025-02-06
3,Steel,3286.00,28.00,0.86%,0.86%,2.75%,-0.73%,-15.40%,2025-02-06
4,Iron Ore CNY,817.50,13.00,1.62%,1.62%,8.57%,4.94%,-13.40%,2025-02-06
5,Lithium,77300,350,-0.45%,-0.45%,2.45%,3.00%,-20.72%,2025-02-06
6,Platinum,1011.30,4.30,-0.42%,-0.51%,5.04%,13.12%,14.98%,2025-02-06
7,Iron Ore,104.81,0.59,-0.56%,3.43%,5.40%,1.16%,-17.73%,2025-02-06
8,Titanium,45.50,0.00,0.00%,0.00%,2.25%,2.25%,-3.43%,2025-02-06
9,HRC Steel,740.00,2.00,-0.27%,1.09%,4.23%,4.37%,-12.53%,2025-02-06


##### Agricultural

In [131]:
table = driver.find_elements(By.XPATH, '/html/body/form/div[4]/div/div/div[3]/div/table/tbody')[0]
rows = table.find_elements(By.TAG_NAME, 'tr')
data = []

# Iterate through each row and extract data
for row in rows:
    columns = row.find_elements(By.TAG_NAME, 'td')
    
    # Extract data from each column based on the specified headers
    Agricultural = columns[0].text.strip().split("\n")[0]
    price = columns[1].text.strip()
    day = columns[2].text.strip()
    day_percent = columns[3].text.strip()
    weekly = columns[4].text.strip()
    monthly = columns[5].text.strip()
    ytd = columns[6].text.strip()
    yoy = columns[7].text.strip()
    date = today
    data.append([Agricultural, price, day, day_percent, weekly, monthly, ytd, yoy, date])
    # Print the extracted data
    print(f"Agricultural: {Agricultural}, price: {price}, Day: {day}, day_percent: {day_percent}, Weekly: {weekly}, Monthly: {monthly}, YTD: {ytd}, YoY: {yoy},  Date: {date}")

df_trading_economics_Agricultural = pd.DataFrame(data, columns=['Agricultural', 'price', 'Day', 'day_percent', 'Weekly', 'Monthly', 'YTD', 'YoY', 'Date'])

Agricultural: Soybeans, price: 1061.75, Day: 4.75, day_percent: 0.45%, Weekly: 1.70%, Monthly: 7.00%, YTD: 6.36%, YoY: -11.34%,  Date: 2025-02-06
Agricultural: Wheat, price: 587.25, Day: 15.00, day_percent: 2.62%, Weekly: 3.66%, Monthly: 8.25%, YTD: 6.48%, YoY: -3.41%,  Date: 2025-02-06
Agricultural: Lumber, price: 585.62, Day: 12.69, day_percent: 2.21%, Weekly: 1.48%, Monthly: 6.18%, YTD: 6.47%, YoY: 5.71%,  Date: 2025-02-06
Agricultural: Palm Oil, price: 4405.00, Day: 73.00, day_percent: 1.69%, Weekly: 2.87%, Monthly: 0.73%, YTD: -0.88%, YoY: 13.62%,  Date: 2025-02-06
Agricultural: Cheese, price: 1.9112, Day: 0.0062, day_percent: 0.33%, Weekly: 0.06%, Monthly: -1.94%, YTD: 0.54%, YoY: 12.29%,  Date: 2025-02-06
Agricultural: Milk, price: 20.32, Day: 0.02, day_percent: -0.10%, Weekly: -0.20%, Monthly: -0.73%, YTD: 8.61%, YoY: 25.20%,  Date: 2025-02-06
Agricultural: Rubber, price: 193.00, Day: 0.80, day_percent: -0.41%, Weekly: -3.16%, Monthly: 4.16%, YTD: -2.23%, YoY: 26.56%,  Date: 20

In [132]:
df_trading_economics_Agricultural

,Agricultural,price,Day,day_percent,Weekly,Monthly,YTD,YoY,Date
0,Soybeans,1061.75,4.75,0.45%,1.70%,7.00%,6.36%,-11.34%,2025-02-06
1,Wheat,587.25,15.00,2.62%,3.66%,8.25%,6.48%,-3.41%,2025-02-06
2,Lumber,585.62,12.69,2.21%,1.48%,6.18%,6.47%,5.71%,2025-02-06
3,Palm Oil,4405.00,73.00,1.69%,2.87%,0.73%,-0.88%,13.62%,2025-02-06
4,Cheese,1.9112,0.0062,0.33%,0.06%,-1.94%,0.54%,12.29%,2025-02-06
5,Milk,20.32,0.02,-0.10%,-0.20%,-0.73%,8.61%,25.20%,2025-02-06
6,Rubber,193.00,0.80,-0.41%,-3.16%,4.16%,-2.23%,26.56%,2025-02-06
7,Orange Juice,438.28,5.07,-1.14%,-8.62%,-13.60%,-11.90%,12.11%,2025-02-06
8,Coffee,403.30,3.37,0.84%,7.22%,26.10%,25.84%,111.49%,2025-02-06
9,Cotton,66.03,0.02,0.04%,-0.29%,-3.62%,-3.42%,-25.37%,2025-02-06


### Riesgo Pais

In [361]:
try:
    driver.quit()
except:
    print("no driver open")       

In [362]:
driver = start_driver()

In [363]:
time.sleep(5)
driver.get("https://bcrdgdcprod.blob.core.windows.net/documents/entorno-internacional/documents/Serie_Historica_Spread_del_EMBI.xlsx")
time.sleep(15)

In [364]:

file_to_rename = None
for filename in os.listdir(source_directory):
    if "Spread" in filename:
        print("its here")
        file_to_rename = filename
        break  

if file_to_rename is None:
    print("No file containing 'Spread' found.")
else:
    
    new_filename = f"{today}_{file_to_rename}"
    old_file_path = os.path.join(source_directory, file_to_rename)
    new_file_path = os.path.join(source_directory, new_filename)
    os.rename(old_file_path, new_file_path)
    print(f"Renamed '{file_to_rename}' to '{new_filename}'.")


its here
Renamed 'Serie_Historica_Spread_del_EMBI.xlsx' to '2025-02-10_Serie_Historica_Spread_del_EMBI.xlsx'.


In [376]:
df_riesgo_pais = pd.read_excel(new_file_path, usecols="A: U", skiprows=1)

In [377]:
df_riesgo_pais

,Fecha,Global,LATINO,REP DOM,Argentina,Bolivia,Brasil,Chile,Colombia,Costa Rica,...,El Salvador,Guatemala,Honduras,México,Paraguay,Perú,Panamá,Uruguay,Venezuela,RD-LATINO
0,2007-10-29 00:00:00,2.10117,2.32541,2.28217,3.50066,NaN,1.74591,1.12264,1.6012,NaN,...,1.69250,NaN,NaN,1.25739,NaN,1.45609,1.57240,2.05193,4.01601,-0.04324
1,2007-10-30 00:00:00,2.09184,2.30653,2.28059,3.40902,NaN,1.73399,1.12886,1.57447,NaN,...,1.66766,NaN,NaN,1.26494,NaN,1.45076,1.56610,2.01736,3.99061,-0.02594
2,2007-10-31 00:00:00,2.01050,2.20068,2.13124,3.11642,NaN,1.65654,1.11809,1.47853,NaN,...,1.58238,NaN,NaN,1.20928,NaN,1.36579,1.45827,1.91474,3.82922,-0.06944
3,2007-11-01 00:00:00,2.12474,2.31998,2.22515,3.28787,NaN,1.78215,1.09782,1.60699,NaN,...,1.67401,NaN,NaN,1.30770,NaN,1.45911,1.57267,1.98080,4.01389,-0.09483
4,2007-11-02 00:00:00,2.20092,2.39996,2.31217,3.40686,NaN,1.85498,1.14779,1.68363,NaN,...,1.74595,NaN,NaN,1.37776,NaN,1.52888,1.65530,2.04029,4.12753,-0.08779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4304,2025-02-03 00:00:00,2.89859,4.10078,1.99453,6.36005,20.51716,2.20482,1.25955,3.26368,1.97482,...,3.82580,2.05131,4.23729,3.16521,1.63632,1.61279,2.86897,0.94961,191.78763,-2.10625
4305,2025-02-04 00:00:00,2.90479,4.08731,1.99544,6.39842,20.26377,2.17381,1.26148,3.29796,1.97185,...,3.80938,1.97395,4.26368,3.17352,1.61984,1.61390,2.81447,0.93847,192.03015,-2.09187
4306,2025-02-05 00:00:00,2.89523,4.07226,1.98486,6.59398,20.39050,2.17020,1.28356,3.3103,2.02007,...,3.83911,2.07362,4.24687,3.14877,1.61270,1.61317,2.87346,0.94311,182.91839,-2.08740
4307,2025-02-06 00:00:00,2.88318,4.06245,2.05165,6.51453,20.47037,2.20954,1.27685,3.32315,2.00531,...,3.76289,2.04647,4.25480,3.14337,1.59482,1.57260,2.90458,0.95363,181.83381,-2.01080


In [378]:
try:
    shutil.move(new_file_path, backup_directory)
    print(f"Moved '{new_filename}' to '{backup_directory}'.")
except Exception as e:
    print(f"Error moving the file: {e}")

Moved '2025-02-10_Serie_Historica_Spread_del_EMBI.xlsx' to 'C:\Users\bd1266\Downloads\Data Download\Back Up'.


In [ ]:
df_ppalesvariables,
df_factores_variacion,
df_agregados_monetarios,
df_depositos_extranjera,
df_tasas_interés_nominal_anual,
saldos_mill_moneda_origen,
otros_indicadores_df,
df_tasas_interes_prestamos_pesos,
df_variacion_base_monetaria,
df_trading_economics_bonds,
df_trading_economics_stocks,
df_trading_economics_currencies,
df_trading_economics_Energy,
df_trading_economics_Metal,
df_trading_economics_Agricultural,
df_riesgo_pais

In [380]:
------------------------------------------------------------------------------------------------------------------------------------------------------